In [1]:
import sys

sys.path.append('../..')

In [2]:
import dill

loaded_data = None

with open('../simulations/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

keys = loaded_data.keys()
keys.sort()
print keys

['N', 'credit', 'randomization', 'underlyings']


In [3]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 15*step

maturity = loaded_data["randomization"]["maturity"]

#Discount

In [4]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [5]:
udlyings = loaded_data["underlyings"]

bm = udlyings[0]
time_grid = bm.time

drift = bm.drifts[0][0]
vol = bm.vols[0][0]

# Derivative

In [6]:
derivatives_nb = 1

In [7]:
from finance.products.european.assetswap import (
    SwapContract,
)

swap_delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(bm, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 =  0.0


# Exposure

In [8]:
from risk.exposures import EuropeanQuantileBrownianExposure

quantile_im = 0.80

exposure = EuropeanQuantileBrownianExposure(swap, discount, drift, vol)

# Indexes stuffs

## All idiosyncratic ids

In [9]:
copula = loaded_data["credit"]["copula"]

all_ids = [list(sub[0])[0] for sub in copula.subsets if len(sub[0]) == 1]
obligors_nb = len(all_ids)

print "Obligors number: %s\n"%obligors_nb
print "All ids (including those that have a zero weight): %s"%all_ids

Obligors number: 125

All ids (including those that have a zero weight): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124]


## Ids of $B$ and $C$

In [10]:
b_id = 113

bc_ids = loaded_data["credit"]["bc_ids"]
c_ids = bc_ids[:]
c_ids.remove(b_id)

c_ids.sort()

print "Bank id: %s\n"%b_id
print "Counterparties ids: %s"%c_ids

Bank id: 113

Counterparties ids: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124]


## Subsets indexes of the copula for $B$ and $C$

In [11]:
armaggeddon_subset_index = 129

In [12]:
bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

b_subsets_indexes = copula.get_indexes_including(b_id)
b_subsets_indexes.remove(armaggeddon_subset_index)

c_subsets_indexes = bc_subsets_indexes[:]
for idx in b_subsets_indexes:
    c_subsets_indexes.remove(idx)
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n"%bc_subsets_indexes
print "Bank subsets indexes: %s\n"%b_subsets_indexes
print "Counterparties subsets indexes: %s"%c_subsets_indexes

Subsets that generated default times: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124, 125, 126, 127, 128]

Bank subsets indexes: [113, 128]

Counterparties subsets indexes: [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124, 125, 126, 127]


# Portfolio construction

In [13]:
import numpy as np

p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [14]:
from finance.portfolio import EquilibratedPortfolio

pos = EquilibratedPortfolio.generate_1_vs_all_positions(b_id, bc_ids, obligors_nb)
positions = np.array(pos*notional).reshape(pos.size, 1)

In [15]:
port = EquilibratedPortfolio(positions, [swap], [exposure])

# VM and IM accounts

In [16]:
from ccp.accounts import Accounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)

In [17]:
# Be carefull, here, as the default models will be 
# used for regulatory computations, we select only 
# the first 125 idiosyncratic models even if 
# we care only about 61 over 125.

default_proba_models = copula.models[0 : obligors_nb]

# Regulatory capital

In [18]:
from risk.basel import RegulatoryCapital

recoveries = [0.4 for i in range(0, obligors_nb)]

regul_cap = RegulatoryCapital(vm_accounts, im_accounts, port, default_proba_models, recoveries, bank_index=b_id)

# Funding and capital parameters

In [19]:
lambda_ = 0.
recov_funding = 1.
k_ = 0.1

c_0 = -.0020

#MC loop

In [20]:
from utils import time_offseter

In [21]:
N = loaded_data["N"]

In [22]:
results_idio = {"cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "dva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "fva": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_ccr": {c: {"sum": 0., "sum2": 0.} for c in c_ids},
                "kva_cva": {c: {"sum": 0., "sum2": 0.} for c in c_ids}}

results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva_ccr": {"sum": 0., "sum2": 0.},
                 "kva_cva": {"sum": 0., "sum2": 0.}}

In [23]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [24]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    zeta_delta = time_offseter(zeta+delta, time_grid, True)
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]
    
    b_min_df_time = default_times[b_df_times_indexes].min()
    
    cva_, dva_, fva_, kccr_, kcva_ = [0. for _ in range(5)]
    
    for c_id in c_ids:
        cp_subsets_indexes = copula.get_indexes_including(c_id)
        c_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in cp_subsets_indexes]
        
        bc_df_times_indexes = np.append(b_df_times_indexes, c_df_times_indexes)
        bc_df_times_indexes = np.sort(np.unique(bc_df_times_indexes))
        
        bc_df_times = default_times[bc_df_times_indexes]
        
        tau = bc_df_times.min()
        bar_tau = min(tau, maturity)
        
        cva__, dva__, fva__, kccr__, kcva__ = [0. for _ in range(5)]
        del bc_df_times
        
        if bar_tau <= zeta:
            continue
            
        inv_pdf = 1./exp_distrib.pdf(zeta)
        discount_ratio = discount(zeta_delta) / discount(zeta)
        mult_factor = inv_pdf*discount_ratio

        # VM computations
        d_pl = [s.price(zeta) for s in [swap]]
        b_vm = port.compute_value(d_pl, from_=b_id, towards_=c_id)
        vm_accounts.put_amounts(b_id, b_vm)
        vm_accounts.put_amounts(c_id, -b_vm)

        # IM computations
        b_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, \
                                     from_=b_id, towards_=c_id)
        c_im = port.compute_exposure(zeta, risk_period=delta, conf_level=quantile_im, \
                                     from_=c_id, towards_=b_id)
        
        im_accounts.put_amounts(b_id, np.maximum(b_im - vm_accounts.amounts[b_id], 0.))
        im_accounts.put_amounts(c_id, np.maximum(c_im - vm_accounts.amounts[c_id], 0.))
        
        
        p_zeta = b_vm.sum()        
        d_pl_zeta_delta = [s.price(zeta_delta) for s in [swap]]
        p_zeta_delta = port.compute_value(d_pl_zeta_delta, from_=b_id, towards_=c_id).sum()        
        # We suppose here that \Delta_{\zeta} is null
        q_zeta_delta = p_zeta_delta
        
        c_min_df_time = default_times[c_df_times_indexes].min()
        
        for df_time_index in bc_df_times_indexes:
            subset_index = bc_subsets_indexes[df_time_index]
            gamma_zeta = copula.gamma(subset_index, zeta)
            
            ####################
            ##### CVA part #####
            ####################
            c_default_indic = df_time_index in c_df_times_indexes
            if not c_default_indic:
                c_default_indic = c_min_df_time <= zeta_delta
                
            if c_default_indic:
                b_collat = p_zeta + im_accounts.get_amount(b_id).sum()
                loss = np.maximum(q_zeta_delta-b_collat, 0.)
                cva__ += gamma_zeta*(1.-recoveries[c_id]*loss)
                
                
            ####################
            ##### DVA part #####
            ####################
            b_default_indic = df_time_index in b_df_times_indexes
            if not b_default_indic:
                b_default_indic = b_min_df_time <= zeta_delta
                
            if b_default_indic:
                c_collat = p_zeta - im_accounts.get_amount(c_id).sum()
                loss = np.maximum(q_zeta_delta-c_collat, 0.)
                dva__ += gamma_zeta*(1.-recoveries[b_id]*loss)
                
                
            ####################
            ##### FVA part #####
            ####################
            b_funded_collat = p_zeta + \
                                im_accounts.get_amount(b_id).sum() - \
                                im_accounts.get_amount(c_id).sum()

            b_gamma_zeta = copula.tot_gamma(zeta, subsets_indexes=b_subsets_indexes)
            
            bar_lambda_zeta = .5*(1.-recoveries[b_id])*b_gamma_zeta
            tilde_lambda_zeta = bar_lambda_zeta - (1.-recov_funding)*b_gamma_zeta
            c_zeta = c_0 + bar_lambda_zeta
            lambda_zeta = lambda_
            
            fva__ += b_gamma_zeta * (c_zeta*b_funded_collat + \
                                     tilde_lambda_zeta*np.maximum(p_zeta-b_funded_collat, 0.) + \
                                     lambda_zeta*np.minimum(p_zeta-b_funded_collat, 0.))
            
            
            ######################
            ##### K_CCR part #####
            ######################
            kccr__ += k_*regul_cap.compute_kccr(c_id, zeta)
            
            
            ######################
            ##### K_CCR part #####
            ######################
            kcva__ += k_*regul_cap.compute_kcva(c_id, zeta)
            
        results_idio['cva'][c_id]["sum"] += (mult_factor*cva__)
        results_idio['cva'][c_id]["sum2"] += (mult_factor*cva__)**2
        
        results_idio['dva'][c_id]["sum"] += (mult_factor*dva__)
        results_idio['dva'][c_id]["sum2"] += (mult_factor*dva__)**2
        
        results_idio['fva'][c_id]["sum"] += (mult_factor*fva__)
        results_idio['fva'][c_id]["sum2"] += (mult_factor*fva__)**2
        
        results_idio['kva_ccr'][c_id]["sum"] += (mult_factor*kccr__)
        results_idio['kva_ccr'][c_id]["sum2"] += (mult_factor*kccr__)**2
        
        results_idio['kva_cva'][c_id]["sum"] += (mult_factor*kcva__)
        results_idio['kva_cva'][c_id]["sum2"] += (mult_factor*kcva__)**2
        
        cva_ += (mult_factor*cva__)
        dva_ += (mult_factor*dva__)
        fva_ += (mult_factor*fva__)
        kccr_ += (mult_factor*kccr__)
        kcva_ += (mult_factor*kcva__)
    
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva_ccr"]["sum"] += kccr_
    results_globs["kva_ccr"]["sum2"] += kccr_**2
    
    results_globs["kva_cva"]["sum"] += kcva_
    results_globs["kva_cva"]["sum2"] += kcva_**2
                    
toc = time.time()

#Results

In [25]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], \
                                                         copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "Confidence level used for IM: %.2f"%quantile_im

Results for 20000 iterations (12695.6849999 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 113 that belongs to the following MO copula subsets:
- [113] with pillars [3 5] and intensity [ 0.00374407  0.00339649]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

Counterparties indexes:
[0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 114, 116, 118, 119, 120, 122, 124]

-----------------------------------------------------

Counterparty index 0 belongs to the following MO copula subses:
- [0] with pillars [3 

In [26]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the individual xVA:\n"
keys = results_idio.keys()
keys.sort()

for k in keys:    
    print "%s:"%k.upper()
    print "---------------------------"
    v = results_idio[k]
    for id_, res_ in v.iteritems():        
        mean__ = res_['sum']/N
        mod_var__ = (res_['sum2']/N - mean__**2) / (N-1.)
        half_inter = z_level*np.sqrt(mod_var__)
        print "Id %s: [%f, %f]"%(id_, mean__-half_inter, mean__+half_inter)
    print

Results of the individual xVA:

CVA:
---------------------------
Id 0: [0.019905, 0.020435]
Id 3: [0.020685, 0.021233]
Id 4: [0.025869, 0.026431]
Id 5: [0.025891, 0.026295]
Id 9: [0.022519, 0.022890]
Id 13: [0.042914, 0.044231]
Id 17: [0.138061, 0.142088]
Id 24: [0.035646, 0.036637]
Id 25: [0.022361, 0.022976]
Id 26: [0.491192, 0.503366]
Id 27: [0.035569, 0.036557]
Id 28: [0.020759, 0.021241]
Id 29: [0.231491, 0.235874]
Id 30: [0.026660, 0.027234]
Id 31: [0.143644, 0.146317]
Id 33: [0.028268, 0.028938]
Id 36: [0.029707, 0.030650]
Id 37: [0.026472, 0.027242]
Id 38: [0.240204, 0.244871]
Id 43: [0.026488, 0.027257]
Id 45: [0.046727, 0.048495]
Id 47: [0.031376, 0.032353]
Id 50: [0.051706, 0.053169]
Id 52: [0.038730, 0.039752]
Id 55: [0.050897, 0.052445]
Id 56: [0.047734, 0.049295]
Id 57: [0.032411, 0.032964]
Id 61: [0.051033, 0.052248]
Id 64: [0.084453, 0.087083]
Id 66: [0.068093, 0.070429]
Id 70: [0.102840, 0.106177]
Id 71: [0.150463, 0.155346]
Id 72: [0.025702, 0.026386]
Id 73: [0.023309

In [27]:
print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)    

Results of the global xVA:

The CVA for the bank lies in [5.215308, 5.307876]
The DVA for the bank lies in [1.003964, 1.023747]
The FVA for the bank lies in [0.000589, 0.000614]
The KVA_CCR for the bank lies in [7.002556, 7.128936]
The KVA_CVA for the bank lies in [2.744705, 2.799761]
